# Sentiment Analysis and Stock-Out Prediction for Modern Contraceptives

This Python project analyzes sentiments from news articles about modern contraceptives (e.g., DMPA-SC and IUD) and predicts potential stock-outs based on consumption trends. It uses APIs to fetch news articles and consumption data and leverages sentiment analysis and regression models for predictions.

## Features
1. Fetches news articles for specified contraceptive methods using NewsAPI.
2. Performs sentiment analysis to classify sentiments as Positive, Negative, or Neutral.
3. Collects consumption data from an external API and predicts future stock-outs using regression.
4. Highlights the method with the most positive sentiment and provides stock-out predictions.

## Technologies Used
- **Python**: Core programming language.
- **Libraries**:
  - `requests` for API interaction.
  - `pandas` for data handling.
  - `TextBlob` for sentiment analysis.
  - `scikit-learn` for regression modeling.

## Setup Instructions

### Prerequisites
1. Python 3.8 or higher installed.
2. API keys for:
   - NewsAPI: [Sign up here](https://newsapi.org/).
   - Consumption Data API: Access credentials from the relevant provider.




In [2]:
!pip install textblob

  Obtaining dependency information for textblob from https://files.pythonhosted.org/packages/02/07/5fd2945356dd839974d3a25de8a142dc37293c21315729a41e775b5f3569/textblob-0.18.0.post0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   - -------------------------------------- 30.7/626.3 kB ? eta -:--:--
   -- ------------------------------------ 41.0/626.3 kB 393.8 kB/s eta 0:00:02
   ------ ------------------------------- 112.6/626.3 kB 819.2 kB/s eta 0:00:01
   ------ ------------------------------- 112.6/626.3 kB 819.2 kB/s eta 0:00:01
   ------ ------------------------------- 112.6/626.3 kB 819.2 kB/s eta 0:00:01
   ------------ ------------------------- 204.8/626.3 kB 734.2 kB/s eta 0:00:01
   -------------- ----------------------- 245.8/626.3 kB 888.8 kB/s eta 0:00:01
   -------------- ----------------------- 245.8/626.3 kB 888.8 kB/s eta 0:00:01
   -------------- ----------------------- 245.8/626.3 kB 888.8 kB/s eta 0:00:01
   ------

1. Import Necessary Libraries 

In [3]:
import requests
import pandas as pd
from textblob import TextBlob
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


3 Fetch News Articles

This function uses the NewsAPI to fetch articles on the modern contraceptives most preferred by Clients and a data range. 

In [ ]:
def fetch_news(api_key, query, from_date, to_date):
    """
    Fetch news articles related to a query using the NewsAPI.

    Parameters:
    - api_key (str): API key for NewsAPI.
    - query (str): Search term (e.g., contraceptive method).
    - from_date (str): Start date for news articles.
    - to_date (str): End date for news articles.

    Returns:
    - list: List of articles with title and description.
    """
    url = f"https://newsapi.org/v2/everything?q={query}&from={from_date}&to={to_date}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        print(f"Error fetching news: {response.status_code}")
        return []


4. Perform Sentiment Analysis

Function: Analyze_sentiment

This function processes news articles, combines their titles and descriptions, and performs sentiment analysis using TextBlob. Sentiments are the categorized as:

         1. Positive(polarity > 0)
         2. Negative(Polarity < 0)
         3. Neutral(Polarity=0)

In [ ]:
def analyze_sentiment(articles):
    """
    Perform sentiment analysis on news articles and classify sentiments.

    Parameters:
    - articles (list): List of articles fetched from NewsAPI.

    Returns:
    - DataFrame: DataFrame with sentiment classification (Positive, Negative, Neutral).
    """
    sentiments = []
    for article in articles:
        text = f"{article.get('title', '')} {article.get('description', '')}"
        sentiment_score = TextBlob(text).sentiment.polarity
        sentiment = (
            "Positive" if sentiment_score > 0 else 
            "Negative" if sentiment_score < 0 else 
            "Neutral"
        )
        sentiments.append(sentiment)
    return pd.DataFrame(sentiments, columns=["Sentiment"])


5. Fetch Consumption Data

Function: Fetch_Consumption Data

This function retrieves consumption data for contraceptive methods from an API. 

In [ ]:
def fetch_consumption_data(api_endpoint, method_name, api_key):
    """
    Fetch consumption data for a given contraceptive method.

    Parameters:
    - api_endpoint (str): Endpoint URL for fetching consumption data.
    - method_name (str): Contraceptive method name (e.g., DMPA-SC, IUD).
    - api_key (str): API key for the consumption data API.

    Returns:
    - DataFrame: DataFrame with consumption data.
    """
    headers = {"Authorization": f"Bearer {api_key}"}
    response = requests.get(f"{api_endpoint}?method={method_name}", headers=headers)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    else:
        print(f"Error fetching consumption data: {response.status_code}")
        return pd.DataFrame()


6. Predict Stock-Out
Function: Predict_stock_out

This function uses linear regression to predict future consumption trends and potention stock-outs based on historical consumption of data 

In [ ]:
def predict_stock_out(consumption_data):
    """
    Predict future consumption and potential stock-outs using a simple regression model.

    Parameters:
    - consumption_data (DataFrame): Historical consumption data.

    Returns:
    - array: Predicted consumption values for the future.
    """
    consumption_data["date"] = pd.to_datetime(consumption_data["date"])
    consumption_data.sort_values("date", inplace=True)
    consumption_data.reset_index(drop=True, inplace=True)
    
    # Prepare data for the model
    consumption_data["Index"] = consumption_data.index
    X = consumption_data[["Index"]]
    y = consumption_data["consumption"]
    
    # Train a linear regression model
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict future consumption
    future_indices = pd.DataFrame(
        {"Index": range(len(consumption_data), len(consumption_data) + 12)}
    )
    future_consumption = model.predict(future_indices)
    return future_consumption


7. Main Execution

This is the main block where we:

     1. Fetch news articles for specified methods.
     2. Perform Sentiment analysis
     3. Fetch Consumption data and predict stock-outs 

In [ ]:
def main():
    """
    Main function to perform sentiment analysis and predict stock-outs.
    """
    # API Configuration
    NEWS_API_KEY = "your_news_api_key_here"
    CONSUMPTION_API_ENDPOINT = "https://example.com/consumption"
    CONSUMPTION_API_KEY = "your_consumption_api_key_here"

    # Parameters
    methods = ["DMPA-SC", "IUD"]
    from_date = "2024-01-01"
    to_date = "2024-11-24"
    
    # Process each method
    sentiments_summary = {}
    consumption_predictions = {}
    
    for method in methods:
        print(f"Processing {method}...")
        
        # Fetch and analyze news articles
        articles = fetch_news(NEWS_API_KEY, method, from_date, to_date)
        if not articles:
            print(f"No news articles found for {method}")
            continue
        
        sentiment_df = analyze_sentiment(articles)
        sentiments_summary[method] = sentiment_df["Sentiment"].value_counts()
        print(f"Sentiment Summary for {method}:\n{sentiments_summary[method]}\n")
        
        # Fetch and predict consumption data
        consumption_data = fetch_consumption_data(CONSUMPTION_API_ENDPOINT, method, CONSUMPTION_API_KEY)
        if not consumption_data.empty:
            future_predictions = predict_stock_out(consumption_data)
            consumption_predictions[method] = future_predictions
        else:
            print(f"No consumption data available for {method}")
    
    # Determine the method with the most positive sentiments
    most_positive_method = max(sentiments_summary, key=lambda m: sentiments_summary[m].get("Positive", 0))
    print(f"\nMethod with the most positive sentiments: {most_positive_method}")
    print(f"\nFuture Consumption Predictions for {most_positive_method}:\n{consumption_predictions.get(most_positive_method, 'No predictions available')}\n")

if __name__ == "__main__":
    main()
